### NutriChat - Text-Based Retrieval Evaluation

This notebook evaluates text-based retrieval approaches:
1. Basic hit rate and MRR metrics
2. Elasticsearch BM25 
3. MinSearch implementation for our nutrition facts dataset.
4. Analysis of retrieval performance across different food categories

In [ ]:
# Import libraries
import json
import requests
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch
import minsearch

In [ ]:
# Load documents with IDs
docs_url = 'https://raw.githubusercontent.com/milanimcgraw/NutriChat/main/nutrition-docs-with-ids.json'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [ ]:
documents[:5]

#### Elasticsearch

In [ ]:
# Initialize Elasticsearch client
es_client = Elasticsearch('http://localhost:9200') 

# Define index settings
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "Food": {"type": "text"},
            "Measure": {"type": "text"},
            "Grams": {"type": "float"},
            "Calories": {"type": "float"},
            "Protein": {"type": "float"},
            "Fat": {"type": "float"},
            "SatFat": {"type": "float"},
            "Fiber": {"type": "float"},
            "Carbs": {"type": "float"},
            "Category": {"type": "keyword"}
        }
    }
}

index_name = "nutrition-facts" 

# Create index
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [ ]:
# Handle trace amounts
def convert_numeric(value):
    if value == 't':
        return 0.1  # Represent trace amounts as 0.1
    try:
        return float(value)
    except:
        return 0.0

# Index documents
for doc in tqdm(documents):
    # Convert trace amounts and ensure numeric fields
    doc['Grams'] = convert_numeric(doc['Grams'])
    doc['Calories'] = convert_numeric(doc['Calories'])
    doc['Protein'] = convert_numeric(doc['Protein'])
    doc['Fat'] = convert_numeric(doc['Fat'])
    doc['Sat.Fat'] = convert_numeric(doc['Sat.Fat'])
    doc['Fiber'] = convert_numeric(doc['Fiber'])
    doc['Carbs'] = convert_numeric(doc['Carbs'])
    
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": [
                            "Food^3",
                            "Measure",
                            "Grams",
                            "Calories",
                            "Protein",
                            "Fat",
                            "SatFat",
                            "Fiber",
                            "Carbs",
                            "Category"
                        ],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "Category": category
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
# Test search
test_results = elastic_search(
    query="How many calories in an apple?",
    category="Fruits A-F"
)
for doc in test_results[:3]:
    print(f"Food: {doc['Food']}")
    print(f"Calories: {doc['Calories']}")
    print(f"Measure: {doc['Measure']}\n")


#### MinSearch 

In [ ]:
# Initialize MinSearch
index = minsearch.Index(
    text_fields=["Food", "Measure", "Category"],
    keyword_fields=["id", "Category"],
    numeric_fields=["Grams", "Calories", "Protein", "Fat", "SatFat", "Fiber", "Carbs"]
)

index.fit(documents)

In [ ]:
def minsearch_search(query, category):
    boost = {
        'Food': 3.0,
        'Calories': 2.0,
        'Protein': 1.5,
        'Category': 1.0
    }
    
    results = index.search(
        query=query,
        filter_dict={'Category': category},
        boost_dict=boost,
        num_results=5
    )
    
    return results

#### Ground Truth 

In [ ]:
# Load ground truth data
gt_url = 'https://raw.githubusercontent.com/milanimcgraw/NutriChat/main/nutrichat-groundtruthdata.csv'
df_ground_truth = pd.read_csv(gt_url)
ground_truth = df_ground_truth.to_dict(orient='records')

In [ ]:
ground_truth[:5]

In [ ]:
df_ground_truth = pd.read_csv('nutrichat-groundtruthdata.csv'')

In [ ]:
ground_truth = df_ground_truth.to_dict(orient='records')

#### Evaluation Functions (and metrics)

In [ ]:
# Calculate initial relevance
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document_id']
    results = minsearch_search(query=q['question'], category=q['category'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

In [ ]:
# Example relevance calculation
example = [
    [True, False, False, False, False], # 1, 
    [False, False, False, False, False], # 0
    [False, False, False, False, False], # 0 
    [False, False, False, False, False], # 0
    [False, False, False, False, False], # 0 
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1 
    [False, False, True, False, False],  # 1/3
    [False, False, False, False, False], # 0
]

# 1 => 1
# 2 => 1 / 2 = 0.5
# 3 => 1 / 3 = 0.3333
# 4 => 0.25
# 5 => 0.2
# rank => 1 / rank
# none => 0

Hit Rate

In [ ]:
# Hit rate
def hit_rate(relevance_total):
    """Calculate hit rate"""
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

MRR

In [ ]:
# MRR
def mrr(relevance_total):
    total_score = 0.0
    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)
    return total_score / len(relevance_total)

In [ ]:
# Test metrics on example
print("Hit rate:", hit_rate(example))
print("MRR:", mrr(example))

In [ ]:
hit_rate(relevance_total), mrr(relevance_total)

#### Evaluations

Evaluate ElasticSearch

In [ ]:
# ElasticSearch Relevance
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document_id']
    results = elastic_search(query=q['question'], category=q['category'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

In [ ]:
# Calculate ElasticSearch metrics
print("\nElasticsearch Results:")
print("Hit rate:", hit_rate(relevance_total))
print("MRR:", mrr(relevance_total))

In [ ]:
hit_rate(relevance_total), mrr(relevance_total)

Evaluate MinSearch

In [ ]:
# MinSearch Relevance
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document_id']
    results = minsearch_search(query=q['question'], category=q['category'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

In [9]:
# Calculate 
print("\nMinSearch Results:")
print("Hit rate:", hit_rate(relevance_total))
print("MRR:", mrr(relevance_total))

In [ ]:
hit_rate(relevance_total), mrr(relevance_total)

#### Comparison

In [ ]:
# Compare results
print("Elasticsearch:", es_results)
print("MinSearch:", ms_results)

In [19]:
# Evaluation function
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)
    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

  0%|          | 0/4627 [00:00<?, ?it/s]

In [ ]:
# Evaluate both approaches
print("\nElasticsearch results:")
print(evaluate(ground_truth, lambda q: elastic_search(q['question'], q['category'])))

print("\nMinSearch results:")
print(evaluate(ground_truth, lambda q: minsearch_search(q['question'], q['category'])))